In [1]:
import os
import shutil
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict
import cv2

# Directorios actuales
base_dir = "dataset/Merged_Dataset_splited_by_subject_v2"  # Cambia si tu base es diferente
splits = ["train", "valid", "test"]

# Directorio de salida
output_base = "dataset/Merged_Dataset_splited_by_subject_for_classification_v2"

extensions = [".jpg", ".jpeg", ".png"]

# Contador de imágenes por clase y split
counter = defaultdict(lambda: defaultdict(int))

def yolo_to_pixel_coords(yolo_box, img_width, img_height):
    class_id, x_center, y_center, width, height = map(float, yolo_box)
    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height

    x1 = int(x_center - width / 2)
    y1 = int(y_center - height / 2)
    x2 = int(x_center + width / 2)
    y2 = int(y_center + height / 2)

    # Clipping to image bounds
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(img_width, x2), min(img_height, y2)

    return int(class_id), x1, y1, x2, y2

for split in splits:
    img_dir = Path(base_dir) / split / "images"
    lbl_dir = Path(base_dir) / split / "labels"

    all_images = []
    for ext in extensions:
        all_images.extend(list(img_dir.glob(f"*{ext}")))

    print(f"\nProcesando {split} ({len(all_images)} imágenes)...")

    for img_path in tqdm(all_images, desc=f"{split}"):
        label_file = lbl_dir / (img_path.stem + ".txt")
        if not label_file.exists():
            continue

        image = cv2.imread(str(img_path))
        if image is None:
            continue
        h, w = image.shape[:2]

        with open(label_file, "r") as f:
            lines = f.readlines()

        for i, line in enumerate(lines):
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            class_id, x1, y1, x2, y2 = yolo_to_pixel_coords(parts, w, h)

            cropped = image[y1:y2, x1:x2]
            if cropped.size == 0:
                continue

            # Carpeta destino
            dest_dir = Path(output_base) / split / str(class_id)
            dest_dir.mkdir(parents=True, exist_ok=True)

            # Nombre único por si hay múltiples objetos en una imagen
            output_name = f"{img_path.stem}_{i}{img_path.suffix}"
            output_path = dest_dir / output_name
            cv2.imwrite(str(output_path), cropped)

            counter[split][str(class_id)] += 1

# Mostrar resumen
print("\n📊 Resumen de imágenes por clase y split:")
for split in splits:
    print(f"\n🔹 {split.upper()}:")
    for class_id in sorted(counter[split]):
        print(f"  Clase {class_id}: {counter[split][class_id]} imágenes")

print("\n✅ Dataset clasificado y recortado correctamente.")


Procesando train (33422 imágenes)...


train: 100%|██████████| 33422/33422 [13:52<00:00, 40.13it/s]



Procesando valid (6288 imágenes)...


valid: 100%|██████████| 6288/6288 [03:26<00:00, 30.40it/s]



Procesando test (3954 imágenes)...


test: 100%|██████████| 3954/3954 [02:09<00:00, 30.62it/s]


📊 Resumen de imágenes por clase y split:

🔹 TRAIN:
  Clase 0: 17323 imágenes
  Clase 1: 16112 imágenes

🔹 VALID:
  Clase 0: 4026 imágenes
  Clase 1: 2262 imágenes

🔹 TEST:
  Clase 0: 2464 imágenes
  Clase 1: 1490 imágenes

✅ Dataset clasificado y recortado correctamente.
